# Charity Funding Predictor
#### Using machine learning and neural networks, predict whether applicants will be successful if funded by Alphabet Soup

#### Charity dataset that capture metadata about each organization:
* **EIN** and **NAME**—Identification columns
* **APPLICATION_TYPE**—Alphabet Soup application type
* **AFFILIATION**—Affiliated sector of industry
* **CLASSIFICATION**—Government organization classification
* **USE_CASE**—Use case for funding
* **ORGANIZATION**—Organization type
* **STATUS**—Active status
* **INCOME_AMT**—Income classification
* **SPECIAL_CONSIDERATIONS**—Special consideration for application
* **ASK_AMT**—Funding amount requested
* **IS_SUCCESSFUL**—Was the money used effectively

## Step 1: Preprocess the Data

In [1]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
# application_df = pd.read_csv("Resources/charity_data.csv")
application_df = pd.read_csv("/content/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# this is the target columns
application_df['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

* What variable(s) are the target(s) for your model?
* What variable(s) are the feature(s) for your model?

### Target variable: IS_SUCCESSFUL; 1 is successful, 0 (unsuccessful)
- IS_SUCCESSFUL              int64

### Feature variables:
- APPLICATION_TYPE          object
- AFFILIATION               object
- CLASSIFICATION            object
- USE_CASE                  object
- ORGANIZATION              object
- STATUS                     int64
- INCOME_AMT                object
- SPECIAL_CONSIDERATIONS    object
- ASK_AMT                    int64


In [5]:
application_df.shape

(34299, 12)

In [6]:
# Drop the non-beneficial ID column 'EIN' .
application_df = application_df.drop(['EIN'], axis=1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
application_df.dtypes

NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [8]:
# Determine the number of unique values in each column.
ctn_unique_values = application_df.nunique()
print(f'Number of unique values in each column : \n{ctn_unique_values}')

Number of unique values in each column : 
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [10]:
# Look at NAME value counts for binning
cnt_name = application_df['NAME'].value_counts()
cnt_name[cnt_name > 10]

PARENT BOOSTER USA INC                                                    1260
TOPS CLUB INC                                                              765
UNITED STATES BOWLING CONGRESS INC                                         700
WASHINGTON STATE UNIVERSITY                                                492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                            408
                                                                          ... 
USA SWIMMING INC                                                            11
SIGMA ALPHA OMEGA CHRISTIAN SORORITY INC                                    11
SOCIETY FOR PRESERVATION & ENCOURMT OF BARBERSHOP QUARTET SINGING AMER      11
SONS OF ITALY IN AMERICA                                                    11
FELLOWSHIP OF CHRISTIAN COWBOYS INC                                         11
Name: NAME, Length: 204, dtype: int64

In [13]:
# Choose a cutoff value and create a list of names to be replaced
# use the variable name `names_to_replace`
names_to_replace = list(cnt_name[cnt_name < 10].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [14]:
# Look at APPLICATION_TYPE value counts for binning
cnt_application_type = application_df['APPLICATION_TYPE'].value_counts()
cnt_application_type[cnt_application_type > 10]

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(cnt_application_type[cnt_application_type < 10].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [17]:
# Look at CLASSIFICATION value counts for binning
cnt_classification = application_df['CLASSIFICATION'].value_counts()
cnt_classification[cnt_classification > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [18]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(cnt_classification[cnt_classification < 10].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [19]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_application_df = pd.get_dummies(application_df)
numeric_application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
# Split our preprocessed data into our features and target arrays
y = numeric_application_df["IS_SUCCESSFUL"] #.values

# Features columns are eveything without the target column
X = numeric_application_df.drop(["IS_SUCCESSFUL"],axis=1) #.values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the model - Optimize the original Model 

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model(hidden_nodes_layer1, hidden_nodes_layer2, hidden_nodes_layer3):
    
    number_input_features = len(X_train_scaled[0])
    
    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(
          tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
    )

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))
    
    # Third hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    nn.summary()
    return nn

## Step 2: Compile, Train, and Evaluate the Model

### Attemp 1: Execute, Compile, Train, and Evaluate the Model

In [23]:
# Attemp 1: Execute the model
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5
nn = create_model(hidden_nodes_layer1,hidden_nodes_layer2, hidden_nodes_layer3)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 2360      
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 5)                 30        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5984 - accuracy: 0.7171
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4872 - accuracy: 0.7816
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4672 - accuracy: 0.7820
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4584 - accuracy: 0.7832
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4532 - accuracy: 0.7851
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4500 - accuracy: 0.7862
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4471 - accuracy: 0.7867
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4450 - accuracy: 0.7869
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4436 - accuracy: 0.7878
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4425 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4491 - accuracy: 0.7840 - 538ms/epoch - 2ms/step
Loss: 0.4490695297718048, Accuracy: 0.7840233445167542


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_1.h5")

### Attemp 2: Execute, Compile, Train, and Evaluate the Model

In [28]:
# Attemp 2 execute the model
number_input_features = len(X_train_scaled[1])
hidden_nodes_layer1 = 16
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10
nn = create_model(hidden_nodes_layer1,hidden_nodes_layer2, hidden_nodes_layer3)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                4720      
                                                                 
 dense_5 (Dense)             (None, 10)                170       
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,011
Trainable params: 5,011
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5456 - accuracy: 0.7415
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4612 - accuracy: 0.7814
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4513 - accuracy: 0.7839
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4464 - accuracy: 0.7846
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4433 - accuracy: 0.7855
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4412 - accuracy: 0.7849
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4393 - accuracy: 0.7855
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4384 - accuracy: 0.7871
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4375 - accuracy: 0.7868
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4367 - accura

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4533 - accuracy: 0.7861 - 462ms/epoch - 2ms/step
Loss: 0.453250527381897, Accuracy: 0.7861224412918091


In [32]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_2.h5")

### Attemp 3: Execute, Compile, Train, and Evaluate the Model

In [33]:
# Attemp 3 execute the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 10
nn = create_model(hidden_nodes_layer1,hidden_nodes_layer2, hidden_nodes_layer3)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 30)                8850      
                                                                 
 dense_9 (Dense)             (None, 15)                465       
                                                                 
 dense_10 (Dense)            (None, 10)                160       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,486
Trainable params: 9,486
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5275 - accuracy: 0.7579
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4595 - accuracy: 0.7839
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4506 - accuracy: 0.7848
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4454 - accuracy: 0.7863
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4428 - accuracy: 0.7882
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4406 - accuracy: 0.7867
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4387 - accuracy: 0.7877
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4376 - accuracy: 0.7897
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4359 - accuracy: 0.7894
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4352 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4557 - accuracy: 0.7859 - 515ms/epoch - 2ms/step
Loss: 0.4557079076766968, Accuracy: 0.785889208316803


In [37]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_3.h5")